### Load tensorflow

In [23]:
import tensorflow as tf

In [24]:
#Reset Default graph - Needed only for Jupyter notebook
tf.reset_default_graph()

### Collect Data

In [25]:
import pandas as pd

In [26]:
data = pd.read_csv('./prices.csv')

### Check all columns in the dataset

In [27]:
data.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

### Drop columns `date` and  `symbol`

In [28]:
data.drop(['date','symbol'],axis=1,inplace=True)

In [29]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set

In [30]:
data=data.head(1000)
data.shape

(1000, 5)

### Divide the data into train and test sets

In [31]:
tf.set_random_seed(100)

In [32]:
X=data.drop('close',axis=1)

In [33]:
y=data.close

In [34]:
from sklearn.model_selection import train_test_split

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

## Building the graph in tensorflow

1.Define input data placeholders

In [36]:
#Input features
x = tf.placeholder(shape=[None,4],dtype=tf.float32, name='x-input')

#Normalize the data
x_n = tf.layers.batch_normalization(x,training=True)

#Actual Prices
y_ = tf.placeholder(shape=[None],dtype=tf.float32, name='y-input')

2.Define Weights and Bias

In [37]:
W = tf.Variable(tf.zeros(shape=[4,1]), name="Weights")
b = tf.Variable(tf.zeros(shape=[1]),name="Bias")

3.Prediction

In [38]:
#We will use normalized data
#y = tf.add(tf.matmul(x,W),b,name='output')
y = tf.add(tf.matmul(x_n,W),b,name='output')

4.Loss (Cost) Function [Mean square error]

In [39]:
loss = tf.reduce_mean(tf.square(y-y_),name='Loss')

5.GradientDescent Optimizer to minimize Loss [GradientDescentOptimizer]

In [40]:
train_op = tf.train.GradientDescentOptimizer(0.003).minimize(loss)

## Execute the Graph for 100 epochs and observe the loss

In [41]:
#Lets start graph Execution
sess = tf.Session()

# variables need to be initialized before we can use them
sess.run(tf.global_variables_initializer())

#how many times data need to be shown to model
training_epochs = 100

In [42]:
for epoch in range(100):
            
    #Calculate train_op and loss
    _, train_loss = sess.run([train_op,loss],feed_dict={x:X_train, 
                                                        y_:y_train})
    
    if epoch % 10 == 0:
        print ('Training loss at step: ', epoch, ' is ', train_loss)

Training loss at step:  0  is  8584.705
Training loss at step:  10  is  8037.824
Training loss at step:  20  is  7552.426
Training loss at step:  30  is  7122.4497
Training loss at step:  40  is  6740.008
Training loss at step:  50  is  6197.0103
Training loss at step:  60  is  3765.5889
Training loss at step:  70  is  3761.349
Training loss at step:  80  is  3760.7395
Training loss at step:  90  is  3760.357


In [43]:
for epoch in range(100):
            
    #Calculate train_op and loss
    _, test_loss = sess.run([train_op,loss],feed_dict={x:X_test, 
                                                        y_:y_test})
    
    if epoch % 10 == 0:
        print ('Training loss at step: ', epoch, ' is ', test_loss)

Training loss at step:  0  is  2699.8074
Training loss at step:  10  is  2633.3752
Training loss at step:  20  is  2633.3076
Training loss at step:  30  is  2633.2546
Training loss at step:  40  is  2633.2136
Training loss at step:  50  is  2633.1824
Training loss at step:  60  is  2633.1638
Training loss at step:  70  is  2633.144
Training loss at step:  80  is  2633.1287
Training loss at step:  90  is  2633.1208


### Get the shapes and values of W and b

Hint: Use sess.run(W) to get W.

In [44]:
sess.run(W)

array([[ 3.2288013],
       [-5.060419 ],
       [-1.6279498],
       [ 1.508581 ]], dtype=float32)

In [45]:
sess.run(b)

array([19.721], dtype=float32)

### Find the Absolute mean square loss difference between training and testing loss.

In [46]:
AMSL=abs(train_loss-test_loss)

In [47]:
AMSL

1127.0227

### Linear Classification using Keras

### Building the simple Neural Network in Keras with one neuron in the dense hidden layer.
#### Use Mean square error as loss function and sgd as optimizer

In [48]:
#Initialize Sequential Graph (model)
model = tf.keras.models.Sequential()

#Normalize input data
model.add(tf.keras.layers.BatchNormalization(input_shape=(4,)))

#Add Dense layer for prediction - Keras declares weights and bias automatically
model.add(tf.keras.layers.Dense(1))
#Create optimizer with non-default learning rate

sgd_optimizer = tf.keras.optimizers.SGD(lr=0.01 , clipnorm=1)
#Compile the model - add Loss and Gradient Descent optimizer

model.compile(optimizer=sgd_optimizer, loss='mse')

### Execute the model

In [49]:
model.fit(X_train, y_train, epochs=50)

Epoch 1/50
670/670 [==============================] - 1s 2ms/step - loss: 8317.4379
Epoch 2/50
670/670 [==============================] - 0s 48us/step - loss: 8171.4192
Epoch 3/50
670/670 [==============================] - 0s 70us/step - loss: 7992.7573
Epoch 4/50
670/670 [==============================] - 0s 73us/step - loss: 7812.1165
Epoch 5/50
670/670 [==============================] - 0s 205us/step - loss: 7613.2953
Epoch 6/50
670/670 [==============================] - 0s 60us/step - loss: 7388.7330
Epoch 7/50
670/670 [==============================] - 0s 63us/step - loss: 7151.5410
Epoch 8/50
670/670 [==============================] - 0s 57us/step - loss: 6921.5614
Epoch 9/50
670/670 [==============================] - 0s 58us/step - loss: 6608.0338
Epoch 10/50
670/670 [==============================] - 0s 104us/step - loss: 6361.4827
Epoch 11/50
670/670 [==============================] - 0s 54us/step - loss: 6087.9007
Epoch 12/50
670/670 [==============================] - 0s 54us

### Classification using Keras 

In [0]:
tf.reset_default_graph()
tf.set_random_seed(42)

### Load the given Iris data using pandas (Iris.csv)

In [50]:
iris=pd.read_csv('Iris.csv',index_col=0)

### Splitting the data into feature set and target set

In [51]:
iris.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
Id,,,,,
1,5.1,3.5,1.4,0.2,Iris-setosa
2,4.9,3.0,1.4,0.2,Iris-setosa
3,4.7,3.2,1.3,0.2,Iris-setosa
4,4.6,3.1,1.5,0.2,Iris-setosa
5,5.0,3.6,1.4,0.2,Iris-setosa


In [52]:
iris.shape

(150, 5)

In [53]:
cols = iris.columns
features = cols[0:4]
target = cols[4]
print(features)
print(target)

Index(['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm'], dtype='object')
Species


In [54]:
data_norm = pd.DataFrame(iris)

In [55]:
import numpy as np
indices = data_norm.index.tolist()
indices = np.array(indices)
np.random.shuffle(indices)
X = data_norm.reindex(indices)[features]
y = data_norm.reindex(indices)[target]

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [56]:
from pandas import get_dummies
y = get_dummies(y)
y.head()

,Iris-setosa,Iris-versicolor,Iris-virginica
Id,,,
99,0,1,0
145,0,0,1
134,0,0,1
144,0,0,1
40,1,0,0


### Divide the dataset into Training and test (70:30)

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=.3)

In [58]:
# Convert to np arrays so that we can use with TensorFlow
X_train = np.array(X_train).astype(np.float32)
X_test  = np.array(X_test).astype(np.float32)
y_train = np.array(y_train).astype(np.float32)
y_test  = np.array(y_test).astype(np.float32)

### Model
Build the model with following layers: <br>
1. First dense layer with 10 neurons with input shape 4 (according to the feature set) <br>
2. Second Dense layer with 8 neurons <br>
3. Output layer with 3 neurons with softmax activation (output layer, 3 neurons as we have 3 classes) <br>
4. Use SGD and categorical_crossentropy loss 

In [59]:
model =  tf.keras.models.Sequential()
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Dense(10, input_shape=(4,), activation='relu', name='fc1'))
model.add(tf.keras.layers.Dense(8, activation='relu', name='fc2'))
model.add(tf.keras.layers.Dense(3, activation='softmax', name='output'))


In [60]:
#Create optimizer with non-default learning rate
sgd_optimizer = tf.keras.optimizers.SGD(lr=0.03)

#Compile the model
model.compile(optimizer=sgd_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

### Fitting the model and predicting 

In [61]:
model.fit(X_train, y_train,validation_data=(X_test,y_test), batch_size=5, epochs=200)

Train on 105 samples, validate on 45 samples
Epoch 1/200
105/105 [==============================] - 1s 7ms/step - loss: 1.0861 - acc: 0.6857 - val_loss: 0.7757 - val_acc: 0.6667
Epoch 2/200
105/105 [==============================] - 0s 383us/step - loss: 0.5605 - acc: 0.8000 - val_loss: 0.5666 - val_acc: 0.6667
Epoch 3/200
105/105 [==============================] - 0s 315us/step - loss: 0.4696 - acc: 0.7905 - val_loss: 0.4310 - val_acc: 0.9556
Epoch 4/200
105/105 [==============================] - 0s 471us/step - loss: 0.3937 - acc: 0.8571 - val_loss: 0.4490 - val_acc: 0.7333
Epoch 5/200
105/105 [==============================] - 0s 398us/step - loss: 0.3627 - acc: 0.8286 - val_loss: 0.5053 - val_acc: 0.6667
Epoch 6/200
105/105 [==============================] - 0s 630us/step - loss: 0.3504 - acc: 0.8476 - val_loss: 0.3218 - val_acc: 0.9333
Epoch 7/200
105/105 [==============================] - 0s 469us/step - loss: 0.3324 - acc: 0.8381 - val_loss: 0.6377 - val_acc: 0.6222
Epoch 8/200


105/105 [==============================] - 0s 419us/step - loss: 0.0746 - acc: 0.9810 - val_loss: 0.1117 - val_acc: 0.9556
Epoch 62/200
105/105 [==============================] - 0s 475us/step - loss: 0.0913 - acc: 0.9810 - val_loss: 0.1129 - val_acc: 0.9556
Epoch 63/200
105/105 [==============================] - 0s 499us/step - loss: 0.1566 - acc: 0.9238 - val_loss: 0.1253 - val_acc: 0.9556
Epoch 64/200
105/105 [==============================] - 0s 326us/step - loss: 0.1119 - acc: 0.9524 - val_loss: 0.1054 - val_acc: 0.9333
Epoch 65/200
105/105 [==============================] - 0s 384us/step - loss: 0.0998 - acc: 0.9714 - val_loss: 0.2392 - val_acc: 0.9333
Epoch 66/200
105/105 [==============================] - 0s 308us/step - loss: 0.0836 - acc: 0.9714 - val_loss: 0.2556 - val_acc: 0.9333
Epoch 67/200
105/105 [==============================] - 0s 440us/step - loss: 0.1277 - acc: 0.9524 - val_loss: 0.1238 - val_acc: 0.9333
Epoch 68/200
105/105 [==============================] - 0s 34

105/105 [==============================] - 0s 285us/step - loss: 0.0767 - acc: 0.9524 - val_loss: 0.2144 - val_acc: 0.9556
Epoch 122/200
105/105 [==============================] - 0s 303us/step - loss: 0.0734 - acc: 0.9619 - val_loss: 0.1618 - val_acc: 0.9556
Epoch 123/200
105/105 [==============================] - 0s 266us/step - loss: 0.0805 - acc: 0.9714 - val_loss: 0.1319 - val_acc: 0.9111
Epoch 124/200
105/105 [==============================] - 0s 435us/step - loss: 0.0761 - acc: 0.9714 - val_loss: 0.2402 - val_acc: 0.9111
Epoch 125/200
105/105 [==============================] - 0s 329us/step - loss: 0.1022 - acc: 0.9333 - val_loss: 0.1354 - val_acc: 0.9111
Epoch 126/200
105/105 [==============================] - 0s 256us/step - loss: 0.0832 - acc: 0.9619 - val_loss: 0.3254 - val_acc: 0.9111
Epoch 127/200
105/105 [==============================] - 0s 375us/step - loss: 0.0577 - acc: 0.9714 - val_loss: 0.5801 - val_acc: 0.7778
Epoch 128/200
105/105 [==============================] 

Epoch 181/200
105/105 [==============================] - 0s 246us/step - loss: 0.0481 - acc: 0.9810 - val_loss: 0.1478 - val_acc: 0.9111
Epoch 182/200
105/105 [==============================] - 0s 259us/step - loss: 0.0651 - acc: 0.9619 - val_loss: 0.2140 - val_acc: 0.9556
Epoch 183/200
105/105 [==============================] - 0s 170us/step - loss: 0.0934 - acc: 0.9524 - val_loss: 0.2026 - val_acc: 0.9556
Epoch 184/200
105/105 [==============================] - 0s 249us/step - loss: 0.1278 - acc: 0.9524 - val_loss: 0.1476 - val_acc: 0.9111
Epoch 185/200
105/105 [==============================] - 0s 154us/step - loss: 0.0810 - acc: 0.9714 - val_loss: 0.2375 - val_acc: 0.9333
Epoch 186/200
105/105 [==============================] - 0s 185us/step - loss: 0.0688 - acc: 0.9524 - val_loss: 0.1598 - val_acc: 0.9111
Epoch 187/200
105/105 [==============================] - 0s 158us/step - loss: 0.0690 - acc: 0.9619 - val_loss: 0.1470 - val_acc: 0.9111
Epoch 188/200
105/105 [==================

In [62]:
print('Neural Network Model Summary: ')
print(model.summary())

Neural Network Model Summary: 
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_2 (Batch multiple                  16        
_________________________________________________________________
fc1 (Dense)                  multiple                  50        
_________________________________________________________________
fc2 (Dense)                  multiple                  88        
_________________________________________________________________
output (Dense)               multiple                  27        
Total params: 181
Trainable params: 173
Non-trainable params: 8
_________________________________________________________________
None


In [63]:
X_test[0:1].shape

(1, 4)

In [64]:
#Actual Label for the first example in test dataset
print(y_test[0])

#Model prediction
prediction = model.predict(X_test[0:1])

#Print prediction
print(prediction[0])

[0. 0. 1.]
[2.8043034e-06 1.6909896e-04 9.9982810e-01]


In [65]:
import numpy as np

#Get predicted number with highest probability
predicted_num = np.argmax(prediction[0])

#Print the number
print(predicted_num)

2


In [66]:
results = model.evaluate(X_test, y_test)

45/45 [==============================] - 0s 347us/step


### Report Accuracy of the predicted values

In [67]:
print('Final test set loss: {:4f}'.format(results[0]))
print('Final test set accuracy: {:4f}'.format(results[1]))

Final test set loss: 0.162908
Final test set accuracy: 0.911111
